#Create the environment

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/My Drive/ESoWC

/content/drive/My Drive/ESoWC


In [4]:
import pandas as pd
import xarray as xr
import numpy as np

#Create the dataset

In [5]:
dataset = xr.open_dataset('Data/05_2019_weather_and_CO.nc')
dataset = dataset.sortby(['latitude','longitude','time'])  
dataframe = dataset.to_dataframe()
dataframe.head()

EMISSIONS_2019  ...  tot_wind
latitude longitude time                                 ...          
43.0     4.0       2019-05-01 00:00:00             NaN  ...  6.296568
                   2019-05-01 01:00:00    1.605808e-13  ...  5.841263
                   2019-05-01 02:00:00    1.405082e-13  ...  5.454147
                   2019-05-01 03:00:00    1.354900e-13  ...  5.150618
                   2019-05-01 04:00:00    1.354900e-13  ...  4.817765

[5 rows x 15 columns]

In [6]:
dataframe['tmp_shift_8']=dataframe.tmp.shift(periods=8)
dataframe.head(10)

EMISSIONS_2019  ...  tmp_shift_8
latitude longitude time                                 ...             
43.0     4.0       2019-05-01 00:00:00             NaN  ...          NaN
                   2019-05-01 01:00:00    1.605808e-13  ...          NaN
                   2019-05-01 02:00:00    1.405082e-13  ...          NaN
                   2019-05-01 03:00:00    1.354900e-13  ...          NaN
                   2019-05-01 04:00:00    1.354900e-13  ...          NaN
                   2019-05-01 05:00:00    1.455263e-13  ...          NaN
                   2019-05-01 06:00:00    2.107622e-13  ...          NaN
                   2019-05-01 07:00:00    5.469782e-13  ...          NaN
                   2019-05-01 08:00:00    8.129401e-13  ...   264.142151
                   2019-05-01 09:00:00    8.330127e-13  ...   263.693481

[10 rows x 16 columns]

In [7]:
dataframe['tot_wind_shift_12']=dataframe.tot_wind.shift(periods=12)
dataframe.head(15)

EMISSIONS_2019  ...  tot_wind_shift_12
latitude longitude time                                 ...                   
43.0     4.0       2019-05-01 00:00:00             NaN  ...                NaN
                   2019-05-01 01:00:00    1.605808e-13  ...                NaN
                   2019-05-01 02:00:00    1.405082e-13  ...                NaN
                   2019-05-01 03:00:00    1.354900e-13  ...                NaN
                   2019-05-01 04:00:00    1.354900e-13  ...                NaN
                   2019-05-01 05:00:00    1.455263e-13  ...                NaN
                   2019-05-01 06:00:00    2.107622e-13  ...                NaN
                   2019-05-01 07:00:00    5.469782e-13  ...                NaN
                   2019-05-01 08:00:00    8.129401e-13  ...                NaN
                   2019-05-01 09:00:00    8.330127e-13  ...                NaN
                   2019-05-01 10:00:00    7.627586e-13  ...                NaN
                   2019-05-01 11:00:00    6.623957e-13  ...                NaN
                   2019-05-01 12:00:00    5.871234e-13  ...           6.296568
                   2019-05-01 13:00:00    5.770871e-13  ...           5.841263
                   2019-05-01 14:00:00    5.871234e-13  ...           5.454147

[15 rows x 17 columns]

In [8]:
dataframe['rel_hum_shift_8']=dataframe.rel_hum.shift(periods=8)
dataframe.head(10)

EMISSIONS_2019  ...  rel_hum_shift_8
latitude longitude time                                 ...                 
43.0     4.0       2019-05-01 00:00:00             NaN  ...              NaN
                   2019-05-01 01:00:00    1.605808e-13  ...              NaN
                   2019-05-01 02:00:00    1.405082e-13  ...              NaN
                   2019-05-01 03:00:00    1.354900e-13  ...              NaN
                   2019-05-01 04:00:00    1.354900e-13  ...              NaN
                   2019-05-01 05:00:00    1.455263e-13  ...              NaN
                   2019-05-01 06:00:00    2.107622e-13  ...              NaN
                   2019-05-01 07:00:00    5.469782e-13  ...              NaN
                   2019-05-01 08:00:00    8.129401e-13  ...         0.000219
                   2019-05-01 09:00:00    8.330127e-13  ...         0.000227

[10 rows x 18 columns]

We create the variables Day and Hours as index

In [9]:
df_to_use_dayhour = dataframe

df_to_use_dayhour['Days'] = df_to_use_dayhour.index.get_level_values("time").day 
df_to_use_dayhour['Hours'] = df_to_use_dayhour.index.get_level_values("time").hour
df_to_use_dayhour.set_index('Days', append=True, inplace=True)
df_to_use_dayhour.set_index('Hours', append=True, inplace  =True)
df_to_use_dayhour = df_to_use_dayhour.reset_index()
df_to_use_dayhour= df_to_use_dayhour.drop(["time"], axis=1)
df_to_use_dayhour = df_to_use_dayhour.set_index(['latitude', 'longitude', 'Days', 'Hours'])

df_to_use_dayhour.head()

EMISSIONS_2019  ...  rel_hum_shift_8
latitude longitude Days Hours                  ...                 
43.0     4.0       1    0                 NaN  ...              NaN
                        1        1.605808e-13  ...              NaN
                        2        1.405082e-13  ...              NaN
                        3        1.354900e-13  ...              NaN
                        4        1.354900e-13  ...              NaN

[5 rows x 18 columns]

#Save the dataset

In [10]:
ds_to_use_dayhour=df_to_use_dayhour.to_xarray()
ds_to_use_dayhour

<xarray.Dataset>
Dimensions:            (Days: 31, Hours: 24, latitude: 33, longitude: 33)
Coordinates:
  * latitude           (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude          (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * Days               (Days) int64 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30 31
  * Hours              (Hours) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
Data variables: (12/18)
    EMISSIONS_2019     (latitude, longitude, Days, Hours) float64 nan ... nan
    u10                (latitude, longitude, Days, Hours) float64 2.632 ... nan
    v10                (latitude, longitude, Days, Hours) float64 -5.72 ... nan
    hcc                (latitude, longitude, Days, Hours) float64 0.0 ... nan
    lcc                (latitude, longitude, Days, Hours) float64 0.04311 ......
    tcc                (latitude, longitude, Days, Hours) float64 0.04311 ......
    ...                 ...
    tcwv               (latitude, longitude, Days, Hours) float64 15.11 ... nan
    tcw                (latitude, longitude, Days, Hours) float64 15.11 ... nan
    tot_wind           (latitude, longitude, Days, Hours) float64 6.297 ... nan
    tmp_shift_8        (latitude, longitude, Days, Hours) float64 nan ... nan
    tot_wind_shift_12  (latitude, longitude, Days, Hours) float64 nan ... nan
    rel_hum_shift_8    (latitude, longitude, Days, Hours) float64 nan ... nan

In [11]:
ds_to_use_dayhour.to_netcdf('Data/05_2019_weather_and_CO_for_model.nc', 'w', 'NETCDF4')